In [1]:
import requests
from bs4 import BeautifulSoup
import re
import random
import firebase_admin
from firebase_admin import credentials, firestore

from recipe_scrapers import scrape_me
import requests
from recipe_scrapers import scrape_html


In [2]:

path = 'supermercado-371012-firebase-adminsdk-lu3fc-48a8c3583d.json'
try:
    app = firebase_admin.get_app()
except ValueError as e:
    cred = credentials.Certificate(path)
    firebase_admin.initialize_app(cred)

firebase_admin.get_app()

db = firestore.client()

In [3]:
def crawl_recipe_links(page):
    # Send the request and get the response
    response = requests.get(page)

    # Parse the response content using Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all elements with the class 'c-teaser__link'
    recipe_elements = soup.find_all(class_='ds-recipe-card__image-wrap ds-teaser-link__image-wrap ds-teaser-link__image-wrap')
    print(len(recipe_elements))

    # Print the text and href attributes of each element
    recipe_links = []
    for html in recipe_elements:
        html = str(html)
        soup = BeautifulSoup(html, 'html.parser')
        
        # find the img tag
        img_tag = soup.find('img')

        # access the src attribute
        src = img_tag['src']
        substring_index = src.index('bilder')
        cutoff_string = src[:substring_index]
        recipe_links.append(cutoff_string)
    return recipe_links


In [49]:
def write_recipe(title, time, ingredients, instructions, image, nutrients, energy, nutri_score, price, emissions):
    doc_ref = db.collection(u'recipes').document(title)
    doc_ref.set({

        u'title': title,
        u'time': time,
        u'ingredients': ingredients,
        u'instructions': instructions,
        u'image': image,
        u'nutrients': nutrients,
        u'energy': energy,
        u'nutri_score' : nutri_score,
        u'price' : price,
        u'emissions' : emissions,


})

In [72]:
units = ['g', 'TL, gehäuft', 'TL, gestr.','TL','ml', 'Msp.', 'n. B.', 'EL, gehäuft', 'EL, gestr.','EL', 'etwas', 'Becher', 'Beet/e', 'Beutel', 'Blatt', 'Blätter', 'Bund', 'Bündel', 'cl', 'cm', 'dicke', 'dl', 'Dose', 'Dose/n', 'dünne', 'Ecke(n)', 'Eimer', 'einige', 'einige Stiele',   'evtl.', 'extra', 'Fässchen', 'Fläschchen', 'Flasche', 'Flaschen',  'Glas', 'Gläser', 'gr. Dose/n', 'gr. Flasche(n)', 'gr. Glas', 'gr. Gläser', 'gr. Kopf', 'gr. Scheibe(n)', 'gr. Stück(e)', 'große', 'großen', 'großer', 'großes', 'halbe', 'Halm(e)', 'Handvoll', 'Kästchen', 'kg', 'kl. Bund', 'kl. Dose/n', 'kl. Flasche/n', 'kl. Glas', 'kl. Gläser', 'kl. Kopf', 'kl. Scheibe(n)', 'kl. Stange(n)', 'kl. Stück(e)', 'kleine', 'kleiner', 'kleines', 'Knolle/n', 'Kopf', 'Köpfe', 'Körner', 'Kugel', 'Kugel/n', 'Kugeln', 'Liter', 'm.-große', 'm.-großer', 'm.-großes', 'mehr', 'mg',  'Paar', 'Paket', 'Pck.', 'Pkt.', 'Platte/n', 'Port.', 'Prise(n)', 'Prisen', 'Prozent %', 'Riegel', 'Ring/e', 'Rippe/n', 'Rispe(n)', 'Rolle(n)', 'Schälchen', 'Scheibe/n', 'Schuss', 'Spritzer', 'Stange/n', 'Stängel', 'Staude(n)', 'Stick(s)', 'Stiel/e', 'Stiele', 'Streifen', 'Stück(e)', 'Tablette(n)', 'Tafel', 'Tafeln', 'Tasse', 'Tasse/n', 'Teil/e',   'Topf', 'Tropfen', 'Tube/n', 'Tüte/n', 'viel', 'wenig', 'Würfel', 'Wurzel', 'Wurzel/n', 'Zehe/n', 'Zweig/e']

In [80]:
#1 TL, gestr. Senf
def ingredient_dict(ingredient_list):
    scraper_ingredients = []
    is_found = False
    # ingredients are like: 1 EL Zucker
    for ingredient in ingredient_list:
        ingred_dict = {}

        
        # check weather one unit is contained in ingredient 
        found_unit_list = []
        for unit in units:
            unit_index = ingredient.find(' '+unit+' ') 
            if unit_index > 0:
                # print('ingredient: ', ingredient)
                # print('unit: ' , unit)
                # print('index: ', unit_index)
                found_unit_list.append(unit)
        if len(found_unit_list) > 0:
            #print('unit index :', unit_index, 'unit', unit)
            unit_index = ingredient.find(' '+found_unit_list[0]+' ') 
            unit_end_index = unit_index + len(found_unit_list[0])+ 1
            key_ingredient = ingredient[unit_end_index:]
            value_unit = ingredient[:unit_end_index]
            ingred_dict['size'] = value_unit.strip()
            ingred_dict['name'] = key_ingredient.strip()
            print('Unit found: ', ingred_dict)
            scraper_ingredients.append(ingred_dict)

        # no unit is found like: 'g' or 'ml'
        else:
            numbers = re.findall(r'\d+', ingredient)
            print('numbers in string ', numbers)
            # check if there are numbers in ingredient in order to extract it like: 1 Apple -> Apple : 1
            if len(numbers)>0:
                # first_num has type str
                first_num = numbers[0]
                new_ingredient = ingredient.replace(first_num+' ', '')
                value_unit = first_num
                ingred_dict['size'] = value_unit.strip()
                ingred_dict['name'] = new_ingredient.strip()
                print('Number found: ', ingred_dict)
                scraper_ingredients.append(ingred_dict)
            # in this case there is no unit like: Apple
            else:
                key_ingredient = ingredient
                value_unit = "not available"
                ingred_dict['size'] = value_unit
                ingred_dict['name'] = key_ingredient
                print('Nothing found: ', ingred_dict)
                scraper_ingredients.append(ingred_dict)
    return scraper_ingredients

In [47]:
def nutri_score():
    return random.randrange(1, 6)

In [46]:
def emissions():
    return random.randrange(1, 6) 

In [8]:
def energy():
    return random.randrange(150, 450, step= 10)

In [16]:
def price():
    return random.randrange(3,9), random.randrange(00,99) 

In [10]:
def instruction_maker(scraper_instructions):
    instruction_list = []
    inst_dict = {}
    inst_dict["step"] = "Anleitung"
    inst_dict["description"] = scraper_instructions
    instruction_list.append(inst_dict)
    return instruction_list
    

In [82]:
def get_recipe_ingredients(url):
    html = requests.get(url).content

    scraper = scrape_html(html=html, org_url=url)

    print(scraper.title())
    # print(scraper.ingredients())
    ingredient_list = ingredient_dict(scraper.ingredients())
    # print(scraper.title())
    instructions_list = instruction_maker(scraper.instructions())

    # how many kcal?
    energy_var = str(energy()) + " kcal"

    # how healthy
    nutri_score_var = str(nutri_score()) + "/5"

    # how emission fiendly
    emissions_var = str(emissions()) + "/5"

    # how much does it costs
    result = price()
    num_len = len(str(result[1]))
    euros = str(result[0])
    cents = str(0)+str(result[1]) if num_len == 1 else str(result[1])
    price_var = euros+ "," + cents
   
    # api_instructions = get_api_instructions(scraper.instructions())

    # check_string(api_instructions)
        
    try:
        write_recipe(scraper.title(),scraper.total_time(), ingredient_list, instructions_list, scraper.image(),scraper.nutrients(), energy_var, nutri_score_var, price_var, emissions_var )
    except ValueError:
        print(f'ERROR: ValueError {scraper.title}')
    except Exception:
        print(f'ERROR: Exception {scraper.title}')
        

# get_recipe_ingredients("https://www.chefkoch.de/rezepte/2318071369564257/Griechischer-Tomatensalat-a-la-Dimitrios.html")

In [85]:
recipe_links = crawl_recipe_links('https://www.chefkoch.de/rs/s0t19p15/Vorspeise-Rezepte.html')

for i, url in enumerate(recipe_links):
    # if i == 3:
    #     break
    try:
        get_recipe_ingredients(url)
    except ValueError:
        get_recipe_ingredients(url)
    except IndexError:
        print('idk why')


59
Blätterteigschnecken Elsässer Art
Unit found:  {'size': '300 g', 'name': 'Frischkäse'}
Unit found:  {'size': '250 g', 'name': 'Speck , gewürfelt'}
Unit found:  {'size': '1 Bund', 'name': 'Frühlingszwiebel(n)'}
numbers in string  ['1']
Number found:  {'size': '1', 'name': 'Zwiebel(n)'}
Unit found:  {'size': '2 Pck.', 'name': 'Blätterteig'}
numbers in string  []
Nothing found:  {'size': 'not available', 'name': 'Salz und Pfeffer'}
ERROR: Exception <bound method SchemaScraperFactory.SchemaScraper.title of <recipe_scrapers._factory.SchemaScraperFactory.SchemaScraper object at 0x000001E0EBDA04C0>>
Feine Forellentaler
Unit found:  {'size': '250 g', 'name': 'Räucherforellenfilet(s)'}
Unit found:  {'size': '125 g', 'name': 'Schlagsahne'}
numbers in string  []
Nothing found:  {'size': 'not available', 'name': 'Salz und Pfeffer , weißer'}
Unit found:  {'size': '1 TL', 'name': 'Sahnemeerrettich'}
Unit found:  {'size': '250 g', 'name': 'Pumpernickel , runde'}
Unit found:  {'size': '½ Bund', 'na